# USING A ROUTE OPTIMIZATION NATIVE APP WITH AISQL


Using Cortex with  either the open route optimization service **API** or self contained **Native App**

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
session = get_active_session()

# Add a query tag to the session. This helps with performance monitoring and troubleshooting
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"oss-deploy-route-optimization-demo", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}

from snowflake.snowpark.functions import *

In [ ]:
CREATE OR REPLACE TABLE VEHICLE_ROUTING_SIMULATOR.REGION_DATA AS SELECT * FROM OVERTURE_MAPS__PLACES.CARTO.PLACE

WHERE ST_GEOHASH(GEOMETRY,2) = 'dp';

SELECT * FROM VEHICLE_ROUTING_SIMULATOR.REGION_DATA LIMIT 10;

In [ ]:
CREATE OR REPLACE TABLE OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.PLACES AS 

SELECT 
    GEOMETRY,
    PHONES[0]::text AS PHONES,
    CATEGORIES:primary::text AS CATEGORY,
    NAMES:primary::text AS NAME,
    ADDRESSES[0] AS ADDRESS,
    COALESCE(categories:alternate:list, ARRAY_CONSTRUCT()) AS ALTERNATE
FROM VEHICLE_ROUTING_SIMULATOR.REGION_DATA
WHERE CATEGORIES:primary IS NOT NULL;

ALTER TABLE VEHICLE_ROUTING_SIMULATOR.PLACES ADD SEARCH OPTIMIZATION ON EQUALITY(ALTERNATE);
ALTER TABLE VEHICLE_ROUTING_SIMULATOR.PLACES ADD SEARCH OPTIMIZATION ON GEO(GEOMETRY);

SELECT * FROM VEHICLE_ROUTING_SIMULATOR.PLACES LIMIT 3;

In [ ]:
sql = '''select * from ( select ST_X(GEOMETRY) LON, ST_Y(GEOMETRY) LAT from OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.PLACES where category = 'hotel') sample (5000 rows)'''

st.map(session.sql(sql))

## Create Base Data Table for App
Now we will focus on putting components together for a streamlit application.  We are aiming to create a dynamic routing analyser.  For this, we will need to generate jobs, vehicles and depots based on any location in the world.  Carto overture maps will help as it contains points of interests anywhere in the world. Cortex complete is also used for a simple 'geocoding' mechanism where a user will type in a key word and cortex will generate the coordinates that matches what central location the user wants to base the routing against

#### 1. Job Template
This template will be used to simulate jobs.  You can change the base values in this notebook - which will effect what sample jobs will be generated.  This template generates a sample of 29 jobs.  The location of these jobs and industry will be dynamic and depends on user interaction with the app.

In [ ]:
CREATE OR REPLACE TABLE OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.JOB_TEMPLATE (

ID INT AUTOINCREMENT PRIMARY KEY,
slot_start INT NOT NULL,
slot_end INT,
skills INT,
product STRING,
status STRING DEFAULT 'active'

);


INSERT INTO OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.JOB_TEMPLATE (slot_start, slot_end, skills, product, status) VALUES
(9, 10, 1, 'pa', 'active'),
(11, 15, 2, 'pb', 'active'),
(16, 18, 2, 'pb', 'active'),
(11, 13, 3, 'pc', 'active'),
(7, 16, 3, 'pc', 'active'),
(10, 15, 2, 'pa', 'active'),
(10, 15, 2, 'pa', 'active'),
(7, 16, 1, 'pa', 'active'),
(9, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 3, 'pc', 'active'),
(11, 15, 2, 'pb', 'active'),
(16, 18, 2, 'pb', 'active'),
(11, 13, 1, 'pa', 'active'),
(7, 16, 1, 'pa', 'active'),
(10, 15, 2, 'pb', 'active'),
(10, 15, 2, 'pb', 'active'),
(7, 16, 1, 'pa', 'active'),
(9, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 2, 'pb', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 1, 'pa', 'active'),
(13, 18, 3, 'pc', 'active');


#### Below are all the categories you could use to simulate distribution points
The app will search through names, categories and alternative categories in order to retrieve the relevant data

In [ ]:
SELECT DISTINCT CATEGORY FROM (
select DISTINCT VALUE::TEXT CATEGORY FROM OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.PLACES, LATERAL FLATTEN (ALTERNATE)

UNION 

SELECT DISTINCT CATEGORY FROM OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.PLACES) WHERE SEARCH((CATEGORY),'food')

### 4. Creating an Industry Lookup

Depending on the selected industry, the app will 'lookup' specifics in order to retrieve/ filter the right information.  It is also used to generate relevant product categories. In this notebook, more industries can be added.

In [ ]:
CREATE OR REPLACE TABLE OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.LOOKUP (
    INDUSTRY STRING,
    PA STRING,
    PB STRING,
    PC STRING,
    IND ARRAY,
    IND2 ARRAY,
    CTYPE ARRAY,
    STYPE ARRAY
);

INSERT INTO OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.LOOKUP (INDUSTRY, PA, PB, PC, IND,IND2, CTYPE, STYPE) 
SELECT
    'healthcare', 
    'flammable', 
    'sharps', 
    'temperature-controlled', 
    ARRAY_CONSTRUCT('hospital health pharmaceutical drug healthcare pharmacy surgical'), 
    ARRAY_CONSTRUCT('supplies warehouse depot distribution wholesaler distributors'), 
    ARRAY_CONSTRUCT('hospital', 'family_practice', 'dentist','pharmacy'), 
    ARRAY_CONSTRUCT('Can handle potentially explosive goods', 'Can handle instruments that could be used as weapons', 'Has a fridge')
UNION ALL
SELECT
    'Food', 
    'Fresh Food Order', 
    'Frozen Food Order', 
    'Non Perishable Food Order', 
    ARRAY_CONSTRUCT('food vegatables meat vegatable'),
    ARRAY_CONSTRUCT('wholesaler warehouse factory processing distribution distributors'), 
    ARRAY_CONSTRUCT('supermarket', 'restaurant', 'butcher_shop'), 
    ARRAY_CONSTRUCT('Can deliver Fresh Food', 'Has a Fridge', 'Premium Delivery')
UNION ALL
SELECT
    'Cosmetics', 
    'Hair Products', 
    'Electronic Goods', 
    'Make-up', 
    ARRAY_CONSTRUCT('hair cosmetics make-up beauty'),
    ARRAY_CONSTRUCT('wholesaler warehouse factory supplies distribution distributors'), 
    ARRAY_CONSTRUCT('supermarket', 'outlet', 'fashion'), 
    ARRAY_CONSTRUCT('Can deliver Fresh Food', 'Has a Fridge', 'Premium Delivery')
UNION ALL
SELECT
    'Beverages', 
    'Alcoholic Beverages', 
    'Carbonated Drinks', 
    'Still Water', 
    ARRAY_CONSTRUCT('beverage drink brewery distillery bottling winery'),
    ARRAY_CONSTRUCT('warehouse distribution depot factory wholesaler'), 
    ARRAY_CONSTRUCT('bar', 'pub', 'restaurant', 'hotel', 'supermarket', 'convenience_store'),
    ARRAY_CONSTRUCT('Age Verification Required', 'Fragile Goods Handler', 'Heavy Load Capacity')

    ;
SELECT * FROM OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.LOOKUP

#### Here is an example of how the lookup workes for healthcare for flammable products

In [ ]:
select * from OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.JOB_TEMPLATE a 

inner join

(select PA from OPENROUTESERVICE_SETUP.VEHICLE_ROUTING_SIMULATOR.LOOKUP where industry = 'healthcare') b

on PRODUCT = 'pa'

## Streamlit App

Now you have setup the functions and standing data tables, you can now have a go at running the streamlit app.  The streamlit application is an example of what a basic route optimization simulator could look like.  Feel free to make any changes as you see fit.

- Within Projects / Streamlit you should see a new application
- Click on the app to run the route optimization simulator